In [1]:
import numpy as np
import duckdb
import pandas as pd
import seaborn as sns

In [2]:
data = pd.read_csv("BDC_2024_Womens_Data.csv")

In [3]:
data.columns

Index(['Date', 'Home Team', 'Away Team', 'Period', 'Clock',
       'Home Team Skaters', 'Away Team Skaters', 'Home Team Goals',
       'Away Team Goals', 'Team', 'Player', 'Event', 'X Coordinate',
       'Y Coordinate', 'Detail 1', 'Detail 2', 'Detail 3', 'Detail 4',
       'Player 2', 'X Coordinate 2', 'Y Coordinate 2'],
      dtype='object')

In [4]:
shot_attempts = duckdb.sql("""
                            SELECT Player, "Detail 1", "Detail 2", "Detail 3", "Detail 4",
                            COUNT (Event)
                            FROM data
                            WHERE Event = 'Goal' OR Event = 'Shot'
                            GROUP BY Player,"Detail 1", "Detail 2", "Detail 3", "Detail 4"
                           """).df()

In [5]:
shot_attempts

,Player,Detail 1,Detail 2,Detail 3,Detail 4,"count(""Event"")"
0,Hilary Knight,Wristshot,On Net,f,f,4
1,Brianne Jenner,Wristshot,On Net,f,f,4
2,Taylor Heise,Snapshot,On Net,f,t,1
3,Alex Carpenter,Wristshot,On Net,f,f,1
4,Megan Keller,Snapshot,Blocked,f,f,1
...,...,...,...,...,...,...
310,Brianne Jenner,Wristshot,Blocked,f,f,1
311,Ella Shelton,Slapshot,Missed,f,t,1
312,Marie-Philip Poulin,Deflection,On Net,f,f,1
313,Kirsten Simms,Wristshot,Missed,t,f,1


In [7]:
shot_attempts = duckdb.sql("""
                            SELECT Player,
                            COUNT (Event)
                            FROM data
                            WHERE Event = 'Goal' OR Event = 'Shot'
                            GROUP BY Player
                           """).df()

In [8]:
# Another possible direction? -- What differentiates a goal vs a shot?
# Who are the good goalies of the game?

# On net but unsuccessful = goalie saved
# One-timer: a shot that occurs when a player meets a teammate's pass with an 
# immediate slapshot, without any attempt to control the puck on their stick.

# Traffic: a condensed area on the ice that has a high concentration of players 

unsuccessful_shot = duckdb.sql("""
                                SELECT Player, Event,
                               "Detail 1", "Detail 2", "Detail 3", "Detail 4"
                               FROM data
                               WHERE Event = 'Shot'
                               """).df()


unsuccessful_shot

successful_shot = duckdb.sql("""
                                SELECT Player, Event,
                               "Detail 1", "Detail 2", "Detail 3", "Detail 4"
                               FROM data
                               WHERE Event = 'Goal'
                               """).df()


# Which types of shot yield more success?
# Hypothesis Test: 
# Null Test: All types of shots have the same chances of yielding a goal.
# Alternative Test: There is a type of shot that yields goals more common

In [9]:

all_shots = duckdb.sql("""
                        SELECT * 
                       FROM unsuccessful_shot
                       FULL JOIN successful_shot
                       ON unsuccessful_shot.Player = successful_shot.Player""")
